In [ ]:
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np

from src.datamodules.datasets.civilcomments import CivilCommentsExtracted
from src.datamodules.datasets.civilcomments import CivilCommentsDataset

load_dotenv()
DATA_DIR = os.getenv("DATA_ROOT_DIR")

spurious_feature_idx = 6

In [ ]:
import numpy as np
from collections import Counter

def downsample(X, y, group):
    unique_groups, counts = np.unique(group, return_counts=True)
    min_count = min(counts)
    
    indices = []
    for g in unique_groups:
        group_indices = np.where(group == g)[0]
        sampled_indices = np.random.choice(group_indices, min_count, replace=False)
        indices.extend(sampled_indices)
    
    indices = np.array(indices)
    return X[indices], y[indices], group[indices]

In [ ]:
import numpy as np

data = CivilCommentsExtracted(
    root_dir=DATA_DIR,
    encoding_extractor="bert"
)

not_extracted_data = CivilCommentsDataset(root_dir=DATA_DIR)
print(not_extracted_data._metadata_fields) # Check metadata fields

metadata = not_extracted_data.metadata_array

metadata_train = metadata[not_extracted_data.split_array == 0].numpy()
metadata_test = metadata[not_extracted_data.split_array == 2].numpy()


train_split = data.get_subset("train")

x_train = train_split[np.arange(len(train_split))][0]
y_train = metadata_train[:, -2] 
c_train = metadata_train[:, spurious_feature_idx]


test_split = data.get_subset("test")

x_test = test_split[np.arange(len(test_split))][0]
y_test = metadata_test[:, -2]
c_test = metadata_test[:, spurious_feature_idx]

In [ ]:
X_balaned, y_balanced, groups_balanced = downsample(x_train, y_train, 2*y_train + c_train)

from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=10000).fit(X_balaned, y_balanced)

model.score()

In [ ]:
# Calculate WGA for each spurious feature

for i in range(15):
    curr_feature = metadata_test[:, i]
    groups = 2 * y_test + curr_feature

    group_acc = []

    for g in np.unique(groups):
        mask = groups == g

        score = model.score(x_test[mask], y_test[mask])

        group_acc.append(score)

    print(f"{i}: worst {min(group_acc) * 100:.2f}, best: {max(group_acc) * 100:.2f}")
    print(f"\t min_group_size = {min(np.unique(groups, return_counts=True)[1])}\n")

print("Label:", model.score(x_test, y_test))

In [ ]:
# Calculate correlations label-feature

for i in range(15):
    print(f"{i}: {np.corrcoef(metadata_train[:, i], y_train)[0, 1]:.4f}", )